# TME 2 

## 1. Introduction

In [1]:
import pandas as pd
import itertools as it
import numpy as np

In [2]:
M1 = [[0,8,7,12], [8,0,9,14], [7,9,0,11], [12,14,11,0]]
M2 = [[0,2,3,8,14,18],[2,0,3,8,14,18],
      [3,3,0,8,14,18],[8,8,8,0,14,18],
      [14,14,14,14,0,18],[18,18,18,18,18,0]]
#UPGMA
M3 = [[0,19,27,8,33,18,13],[19,0,31,18,36,1,13],
          [27,31,0,26,41,32,29],[8,18,26,0,31,17,14],
          [33,36,41,31,0,35,28],[18,1,32,17,35,0,12],
          [13,13,29,14,28,12,0]]
#Neighbor Joining
M4 = [[0,2,4,6,6,8],[2,0,4,6,6,8],
          [4,4,0,6,6,8],[6,6,6,0,4,8],
          [6,6,6,4,0,8],[8,8,8,8,8,0]]

M1D = pd.DataFrame(M1)
M2D = pd.DataFrame(M2)
UPGMAD = pd.DataFrame(M3) 
NJD = pd.DataFrame(M4)

M1D
UPGMAD

,0,1,2,3,4,5,6
0,0,19,27,8,33,18,13
1,19,0,31,18,36,1,13
2,27,31,0,26,41,32,29
3,8,18,26,0,31,17,14
4,33,36,41,31,0,35,28
5,18,1,32,17,35,0,12
6,13,13,29,14,28,12,0


###### Question 1

##### Matrice additive

Une est matrice additive si et seulement si la condition des 4 points est vraie pour tous les quartets i,j,k et l où 1<=i,j,k,l<=n).
La condition des 4 points est remplie quand Dij + Dkl <= max(Dik+Djl,Dil+Djk).

In [3]:
def estadditive(M):
    comb=it.combinations(range(len(M)),4)
    for groupe in comb:
        print(groupe)
        i,j,k,l=groupe
        if not (M[i,j]+M[k,l]<=max(M[i,k]+M[j,l],M[i,l]+M[j,k])):
            return False
    return True

In [4]:
M1 = [[0,8,7,12], [8,0,9,14], [7,9,0,11], [12,14,11,0]]
M1 = np.array(M1)
print(estadditive(M1))

(0, 1, 2, 3)
True


##### Ultramétricité

Une matrice est dite ultramétrique quand, pour tout choix de 3 points i,j,k au moins deux distances sont égales : Dij et Djk.

On a alors mathématiquement : Dij = Djk >Dik

Une matrice ultramétrique est forcément additive.

In [5]:
def estultra(M):
    comb=it.combinations(range(len(M)),3)
    for groupe in comb:
        i,j,k=groupe
        if not(M[i,k]<=max(M[i,j],M[j,k])):
            return False
    return True

In [6]:
M1 = np.array(M1)
print(estultra(M1))

False


###### Question 2

In [7]:
def cluster(M,i):            #i = l'espèce en question
    return np.sum(M[i])

In [8]:
def groupe(M):
    L=[]
    for k in range(len(M)):
        L.append(cluster(M,k))
    return L

In [9]:
print(cluster(M1,1))
print(groupe(M1))

31
[27, 31, 27, 37]


## 2. UPGMA

###### Question 1 

Le format Newick est un format de fichier utilisé en phylogénie pour décrire les relations phylogénétiques entre différents clusters.
Par exemple si on a un ancêtre commun entre A et B puis entre AB et C on écrira sous ce format :

(C : longueur de la branche (A : longueur, B : longueur) : longueur restante de la branche)

L'extension de ce format est .nwk

###### Question 2

In [10]:
def UPGMA(M):
    Alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWYZ'
    M1 = np.copy(M)
    Cardinal = [1 for k in range(len(M1))]
    Clusters = [Alphabet[k] for k in range(len(M1))]
    Newick = ''
    for i in range(len(M1)):
        for j in range(len(M1[i])):
            if i==j:
                M1[i,j] = 1000
    while len(M1)>1:
        Indice = np.argmin(M1)
        i,j = np.unravel_index(np.argmin(M1), M1.shape)  #trouve la ligne et la colonne correspondant au argmin
        d = M1[i,j]/2
        for l in range(len(M1)):
            if l!=i and l!=j:
                M1[i,l] = (M1[i,l]*Cardinal[i]+M1[j,l]*Cardinal[l])/(Cardinal[i]+Cardinal[l])
                M1[l,i] = M1[i,l]
        Cardinal[i]+=Cardinal[j]
        del Cardinal[j]
        Newick+=Clusters[i]+Clusters[j]
        del Clusters[j]
        print(Newick,i,j)
        M1 = np.delete(M1,j,0)
        M1 = np.delete(M1,j,1)
    return M1

In [11]:
M3 = np.array(M3,dtype=float)
UPGMAD
print(UPGMA(M3))

BF 1 5
BFAD 0 3
BFADBG 1 4
BFADBGAB 0 1
BFADBGABAC 0 1
BFADBGABACAE 0 1
[[1000.]]


In [12]:
UPGMAD = pd.DataFrame(UPGMA(M3))
UPGMAD

BF 1 5
BFAD 0 3
BFADBG 1 4
BFADBGAB 0 1
BFADBGABAC 0 1
BFADBGABACAE 0 1


,0
0,1000.0


## 3. NJ

##### Question 1

In [13]:
def U(M,i,N):      #i est l'indice sur les lignes représente un cluster donné
    u=0
    n=len(M)
    for j in range (n):
        u+=M[i,j]/(N-2)
    return u

In [14]:
print(U(M3,0,7))
print(M3)

23.6
[[ 0. 19. 27.  8. 33. 18. 13.]
 [19.  0. 31. 18. 36.  1. 13.]
 [27. 31.  0. 26. 41. 32. 29.]
 [ 8. 18. 26.  0. 31. 17. 14.]
 [33. 36. 41. 31.  0. 35. 28.]
 [18.  1. 32. 17. 35.  0. 12.]
 [13. 13. 29. 14. 28. 12.  0.]]


##### Question 2

In [15]:
def Q(M,i,j,n):
    Ui=U(M,i,n)
    Uj=U(M,j,n)
    return M[i,j]-Ui-Uj

In [16]:
print(Q(M3,1,5,7))

-45.6


##### Question 3

In [17]:
def NJ(M):
    Alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWYZ'
    M1 = np.copy(M)
    N=len(M)
    Clusters = [Alphabet[k] for k in range(N)]
    for k in range(len(M1)):
        M1[k,k]=1000
    while len(M1)>1:
        n = len(M1)
        MQ = np.zeros((n,n),dtype='float')
        for k in range(n):
            MQ[k,k] = 1000
        comb = it.combinations(range(n),2)
        for groupe in comb:
            i,j = groupe
            MQ[i,j] = Q(M1,i,j,N)
            MQ[j,i] = Q(M1,i,j,N)
        i,j = np.unravel_index(np.argmin(MQ), MQ.shape)
        Ui = U(M1,i,N)
        Uj = U(M1,j,N)
        d = M1[i,j]
        Di = (d+(Ui-Uj))/2
        Di = (d+(Uj-Ui))/2
        for l in range(len(M1)):
            if l!=i and l!=j:
                M1[i,l] = (M1[i,l]+M1[j,l]-d)/2
                M1[l,i] = M1[i,l]
        M1 = np.delete(M1,j,0)
        M1 = np.delete(M1,j,1)
    return M1

In [18]:
print(NJ(M3))

[[1000.]]
